In [1]:
from scrapy import signals
from scrapy.utils.project import get_project_settings
from scrapy.crawler import CrawlerProcess
from scrapy.xlib.pydispatch import dispatcher
from multiprocessing.queues import Queue
import scrapy
import multiprocessing
import datetime
import collections
import urllib
import collections
from scrapy import Request as Request

/home/jun/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: ScrapyDeprecationWarning: Importing from scrapy.xlib.pydispatch is deprecated and will no longer be supported in future Scrapy versions. If you just want to connect signals use the from_crawler class method, otherwise import pydispatch directly if needed. See: https://github.com/scrapy/scrapy/issues/1762


In [2]:
business_data=collections.namedtuple('business_data','link,title,status')

In [3]:
class CrawlerWorker(multiprocessing.Process):

    def __init__(self, spider, result_queue):
        multiprocessing.Process.__init__(self)
        self.result_queue = result_queue

        self.crawler = CrawlerProcess(get_project_settings())
        #if not hasattr(project, 'crawler'):
        #    self.crawler.install()
        #self.crawler.configure()

        self.items = []
        self.spider = spider
        dispatcher.connect(self._item_passed, signals.item_passed)

    def _item_passed(self, item):
        self.items.append(item)

    def run(self):
        self.crawler.crawl(self.spider)
        self.crawler.start()
        self.crawler.stop()
        self.result_queue.put(self.items)

In [6]:
class CanberraBusinessSpider(scrapy.Spider):
    name = "CanberraBusinessSpider"
    allowed_domains = ["www.seekbusiness.com.au"]
    start_urls = ['https://www.seekbusiness.com.au/businesses-for-sale/in-canberra-act-2601?rad=50']

    def parse(self, response):
        next_page=response.xpath('//a[contains(@class,"next")]/@href').extract()[0]
        next_page_link=urllib.basejoin(response.url,next_page)
        #print(next_page_link)
        yield Request(next_page_link,callback=self.parse)
        
        business_tags=response.xpath('//a[contains(@href,"business-listing") and count(@*)=1]')
        for invidual_business in business_tags:
            title=invidual_business.xpath('text()').extract()[0]
            link=invidual_business.xpath('@href').extract()[0]
            full_link=urllib.basejoin(response.url,link)
            yield {title:business_data(full_link,title,'not yet')}
            #business={'title':title,'link':full_link}
            #print(business)
        #print(business_tags)

In [7]:
def main():
    result_queue = Queue()
    crawler = CrawlerWorker(CanberraBusinessSpider(), result_queue)
    crawler.start()
    for item in result_queue.get():
        print item
        
if __name__=="__main__":
    main()

2017-01-18 13:45:32 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-18 13:45:32 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-18 13:45:32 [scrapy] INFO: Overridden settings: {}
2017-01-18 13:45:32 [scrapy] INFO: Overridden settings: {}
2017-01-18 13:45:33 [scrapy] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2017-01-18 13:45:33 [scrapy] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2017-01-18 13:45:33 [scrapy] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHe

{u'CafeCoffeeShop - Canberra': business_data(link=u'https://www.seekbusiness.com.au/business-listing/cafecoffeeshop-canberra/265305?s=3103164', title=u'CafeCoffeeShop - Canberra', status='not yet')}
{u"Mo's Mobiles is looking for Licensed Dealers to operate existing Vodafone stores": business_data(link=u'https://www.seekbusiness.com.au/business-listing/mos-mobiles-is-looking-for-licensed-dealers-to-operate-existing-vodafone-stores/281849?s=3103164', title=u"Mo's Mobiles is looking for Licensed Dealers to operate existing Vodafone stores", status='not yet')}
{u'Gozleme King Franchise : Turkish Street Food & Cafe': business_data(link=u'https://www.seekbusiness.com.au/business-listing/gozleme-king-franchise-turkish-street-food-cafe/270622?s=3103164', title=u'Gozleme King Franchise : Turkish Street Food & Cafe', status='not yet')}
{u'Healthy Start Civic': business_data(link=u'https://www.seekbusiness.com.au/business-listing/healthy-start-civic/258794?s=3103164', title=u'Healthy Start Civic